<a href="https://colab.research.google.com/github/ninenine-9/legaldetainment/blob/main/HierarchicalClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I have added some comments such as `# 📋 for cell K` for you to know what goes where in the spreadsheet

There's 2 things you can play with:
- dataset size → see data loading section
- model → see `model_name` variable in Section 2

✏️ Make sure to edit the file name in the output path (section 5)

# 0️⃣ Pre-requisites

## ➊ Imports

In [63]:
import pandas as pd
import torch
import os
import time
from google.colab import drive
from transformers import pipeline
from sklearn.metrics import accuracy_score, cohen_kappa_score, classification_report, confusion_matrix

print("✔️ Imports done")

✔️ Imports done


## ➋ Data retrieval

In [64]:
%cd /content

!git clone https://github.com/ninenine-9/legaldetainment.git

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!cp -r /content/legaldetainment /content/drive/MyDrive/

print("✔️ Git Repository successfully cloned")

/content
fatal: destination path 'legaldetainment' already exists and is not an empty directory.
Mounted at /content/drive
✔️ Git Repository successfully cloned


## ➌ Data loading
`legaldetainment_347p_ALL.xlsx` contains our human-coded data. Below is an explanation of the spreadsheet's various sheets:
- `data` = the full dataset (minus `removed`)
- `removed` = participants I've removed because of inconclusive coding
- `ALL(10)` = a sample of 10 participants

`legaldetainment_blankdata.xlsx` contains our blank data. It contains Pno, Qual and the relevant column headers for the codes (but the content of these columns is blank). Below is an explanation of the spreadsheet's various sheets:
- `ALL` = the full dataset (minus `removed`)
- `ALL(10)` = a sample of 10 participants
- `ALL(20)` = a sample of 20 participants
- etc..

⚠️ YOU SHOULD ALWAYS HAVE MATCHING SHEETS BETWEEN THE BLANK DATASET AND THE HUMAN-CODED DATA!
For example, if you create a new tab/sheet for testing 40 participants, you must have a sheet in both the full dataset and human-coded data with corresponding participants (Pno).


---


_example_

`legaldetainment_347p_ALL.xlsx, sheet = ALL(3)`
| Pno | Qual |
|:-----------|:------------:|
| 200 | Lorem ipsum  |
| 35 | dolor sit amet  |
| 298 | Duis aute irure  |

`legaldetainment_blankdata.xlsx, sheet = ALL(3)`
| Pno | Qual |
|:-----------|:------------:|
| 200 | Lorem ipsum  |
| 35 | dolor sit amet  |
| 298 | Duis aute irure  |

`Same Pnos in both datasets ✅`


❔ Still unsure? the last block of this section can check this for you


In [65]:
data = pd.read_excel("/content/legaldetainment/legaldetainment_blankdata.xlsx", sheet_name = 'ALL(10)') # 📋 for cell H & G

data = data.dropna(subset=["Pno"])
print(f"Dataset size: {data.shape}")

humandata = pd.read_excel("/content/legaldetainment/legaldetainment_347p_ALL.xlsx", sheet_name = 'ALL(10)') # 📋 for cell I

humandata = humandata.dropna(subset=["Pno"])
humandata = humandata.replace({0: "No", 1: "Yes", "0": "No", "1": "Yes"})
humandata = humandata.rename(columns={
    "NoInformation/Evidence": "NoInformation_Evidence",
    "Unclassified/Other": "Unclassified_Other"
})

print("➋ Data loading done")


Dataset size: (10, 8)
➋ Data loading done


### Pno checker

In [66]:
# Compare Pnos
humandata_pnos = set(humandata["Pno"])
data_pnos = set(data["Pno"])

if humandata_pnos == data_pnos:
    print("Same Pnos in both datasets ✅")
else:
    print("Pnos mismatch ❌")
    missing_in_data = humandata_pnos - data_pnos
    missing_in_humandata = data_pnos - humandata_pnos
    if missing_in_data:
        print("Missing in data:", missing_in_data)
    if missing_in_humandata:
        print("Missing in humandata:", missing_in_humandata)


Same Pnos in both datasets ✅


# 1️⃣ Code definitions

In [67]:
code_definitions = {
    "GuiltMoreLikely": "The participant states that, because the judge denied the defendant bail, the judge must know something about the defendant, such as that  they don’t have an alibi, or a criminal history, which means they are more likely to be found guilty at the main trial.",
    "GuiltLessLikely": "The participant states that, because the judge granted the defendant bail, the judge must know something about the defendant, such as that they do have an alibi, or have no criminal history, which means they are more likely to be found innocent at the main trial.",
    "NoInformation_Evidence": "The participant states that there is no / not enough information / evidence to make the judgement asked of them.",
    "InnocentUntilProvenGuilty": "The participant either explicitly says ‘innocent until proven guilty’, says something similar e.g. ‘innocent until guilty’ or expresses this sentiment in another way e.g. that none of the information matters until they are proven guilty so we shouldn’t prejudge.",
    "Confound": "Participant states that it will be the same judge at the main trial as the start AND/OR the participant states that because of the detainment / lack of detainment, the judge may be e.g. biased against them or more likely to find them guilty / not guilty.",
    "Unclassified_Other": "Not one of the previous codes are applicable – either something else or can’t discern thought process at all.",
}

# Define which codes are in the mutually exclusive cluster
cluster_codes = ["GuiltMoreLikely", "GuiltLessLikely", "NoInformation_Evidence"]
independent_codes = ["InnocentUntilProvenGuilty", "Confound"]
all_codes = cluster_codes + independent_codes + ["Unclassified_Other"]

print("➌ Code definitions loaded")

➌ Code definitions loaded


# 2️⃣ Model loading

In [68]:
device = 0 if torch.cuda.is_available() else -1
print(f"🔍 Using: {'GPU' if torch.cuda.is_available() else 'CPU'}")

model_name = "google/flan-t5-large"  # ✏️ + 📋 for cell B

pipe = pipeline("text2text-generation", model=model_name, device=device) # 📋 for cell a

print("➍ Model loading done")

🔍 Using: CPU


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


➍ Model loading done


# 3️⃣ Classification functions

In [75]:
def classify_binary(qual_text: str, code_name: str, code_desc: str) -> str:
    """
    Simple yes/no classification for independent codes
    """
    prompt = f"""Does this response show the following?

{code_desc}

Response: "{qual_text.strip()}"

Answer only Yes or No:"""

    try:
        output = pipe(prompt, max_new_tokens=5, do_sample=False, truncation=True)[0]["generated_text"]
        output_lower = output.lower().strip()

        if "yes" in output_lower:
            return "Yes"
        elif "no" in output_lower:
            return "No"
        else:
            print(f"    ⚠️ Unclear output for {code_name}: '{output}' - defaulting to No")
            return "No"
    except Exception as e:
        print(f"    ⚠️ Error on {code_name}: {e}")
        return "Error"


def classify_cluster(qual_text: str) -> dict:
    """
    Classify the mutually exclusive cluster (GuiltMoreLikely, GuiltLessLikely, NoInformation_Evidence)
    Returns dict with Yes for one code (or none)
    """
    prompt = f"""This response is about how pre-trial detention affects perceptions of guilt. Which ONE category applies, or NONE?

1. GuiltMoreLikely - {code_definitions['GuiltMoreLikely']}
2. GuiltLessLikely - {code_definitions['GuiltLessLikely']}
3. NoInformation_Evidence - {code_definitions['NoInformation_Evidence']}
4. NONE of the above

Response: "{qual_text.strip()}"

Answer with only the number (1, 2, 3, or 4):"""

    try:
        output = pipe(prompt, max_new_tokens=50, do_sample=False, truncation=True)[0]["generated_text"]
        output_stripped = output.strip()

        # Initialize all cluster codes as No
        result = {code: "No" for code in cluster_codes}

        # Set the selected one to Yes
        if "1" in output_stripped:
            result["GuiltMoreLikely"] = "Yes"
            print(f"    Cluster: GuiltMoreLikely")
        elif "2" in output_stripped:
            result["GuiltLessLikely"] = "Yes"
            print(f"    Cluster: GuiltLessLikely")
        elif "3" in output_stripped:
            result["NoInformation_Evidence"] = "Yes"
            print(f"    Cluster: NoInformation_Evidence")
        elif "4" in output_stripped:
            print(f"    Cluster: NONE")
        else:
            print(f"    ⚠️ Unclear cluster output: '{output}' - defaulting to NONE")

        return result

    except Exception as e:
        print(f"    ⚠️ Error on cluster classification: {e}")
        return {code: "Error" for code in cluster_codes}


def classify_hierarchical(qual_text: str, pno: int) -> dict:
    """
    Main hierarchical classification function
    Step 1: Classify cluster (mutually exclusive)
    Step 2: Classify independent codes
    Step 3: Apply Unclassified_Other rule
    """
    print(f"  Processing Pno {pno}...")

    # Step 1: Cluster classification
    result = classify_cluster(qual_text)

    # Step 2: Independent codes
    for code_name in independent_codes:
        result[code_name] = classify_binary(qual_text, code_name, code_definitions[code_name])
        print(f"    {code_name}: {result[code_name]}")

    # Step 3: Unclassified_Other rule
    # If ANY other code is Yes, then Unclassified_Other = No
    # If ALL other codes are No, then Unclassified_Other = Yes
    any_yes = any(result[code] == "Yes" for code in cluster_codes + independent_codes)
    result["Unclassified_Other"] = "No" if any_yes else "Yes"
    print(f"    Unclassified_Other: {result['Unclassified_Other']}")

    return result

# 4️⃣ Main Classification Loop

In [76]:
results = []
start_time = time.time()

print(f"\nStarting hierarchical classification for {len(data)} responses...\n")

for i, row in data.iterrows():
    pno = row["Pno"]
    qual_text = str(row["Qual"]).strip()

    if not qual_text or qual_text == "nan":
        print(f"Skipping Pno {pno} (no text)")
        continue

    # Classify using hierarchical approach
    classification = classify_hierarchical(qual_text, pno)

    # Build result row
    result = {"Pno": pno, "Qual": qual_text}
    for code in all_codes:
        result[f"{code}_LLM"] = classification[code]

    results.append(result)

    # Progress indicator
    if (i + 1) % 10 == 0:
        elapsed = time.time() - start_time
        avg_time = elapsed / (i + 1)
        remaining = avg_time * (len(data) - i - 1)
        print(f"\n✓ Progress: {i+1}/{len(data)} ({(i+1)/len(data)*100:.1f}%)")
        print(f"  Time elapsed: {elapsed:.1f}s | Estimated remaining: {remaining:.1f}s\n")

elapsed = time.time() - start_time
print(f"\n➎ Classification completed in {elapsed:.2f} seconds") # 📋 for cell K
print(f"Average time per response: {elapsed/len(data):.2f}s")


Starting hierarchical classification for 10 responses...

  Processing Pno 15...
    ⚠️ Unclear cluster output: 'GuiltMoreLikely' - defaulting to NONE
    InnocentUntilProvenGuilty: Yes
    Confound: No
    Unclassified_Other: No
  Processing Pno 23...
    Cluster: GuiltLessLikely
    InnocentUntilProvenGuilty: Yes
    Confound: No
    Unclassified_Other: No
  Processing Pno 55...
    Cluster: GuiltMoreLikely
    InnocentUntilProvenGuilty: No
    Confound: No
    Unclassified_Other: No
  Processing Pno 59...
    ⚠️ Unclear cluster output: 'NoInformation_Evidence' - defaulting to NONE
    InnocentUntilProvenGuilty: No
    Confound: No
    Unclassified_Other: Yes
  Processing Pno 66...
    ⚠️ Unclear cluster output: 'NONE' - defaulting to NONE
    InnocentUntilProvenGuilty: No
    Confound: No
    Unclassified_Other: Yes
  Processing Pno 121...
    ⚠️ Unclear cluster output: 'NONE' - defaulting to NONE
    InnocentUntilProvenGuilty: No
    Confound: Yes
    Unclassified_Other: No
  Proc

# 5️⃣ Saving outputs

In [77]:
output_path = "/content/drive/MyDrive/legaldetainment_results_t5large_50t.csv" # ✏️ + 📋 for cell J
output_df = pd.DataFrame(results)
os.makedirs(os.path.dirname(output_path), exist_ok=True)
output_df.to_csv(output_path, index=False)
print(f"➏ Results saved to {output_path}")

➏ Results saved to /content/drive/MyDrive/legaldetainment_results_t5large_50t.csv


# 6️⃣ Evaluation

In [83]:
# humandata = pd.read_excel("/content/drive/MyDrive/legaldetainment_347p_ALL.xlsx", sheet_name = 'ALL(10)')
# data = pd.read_excel("/content/drive/MyDrive/legaldetainment_blankdata.xlsx", sheet_name = 'ALL(10)')

# print(data.columns)

# print(humandata.columns)
# Convert 0/1 to No/Yes
humandata = humandata.replace({'0': 'No', '1': 'Yes'})

# Compare human and model coding
eval_df = humandata[["Pno"] + all_codes].merge(
    output_df[["Pno"] + [f"{code}_LLM" for code in all_codes]],
    on="Pno",
    how="inner"
)

# Boolean: all codes match for that Pno
eval_df["all_match"] = eval_df.apply(
    lambda row: all(row[code] == row[f"{code}_LLM"] for code in all_codes),
    axis=1
)

# Count matches
total = len(eval_df)
matches = eval_df["all_match"].sum()
percentage = matches / total * 100 if total else 0

print(f"\nTotal Pnos compared: {total}")
print(f"Matching on all codes: {matches} ({percentage:.1f}%)") # 📋 for cell N & O

# Print Pnos that match on all codes
matching_pnos = eval_df[eval_df['all_match'] == True]['Pno'].tolist()
print(f"Pnos matching on all codes: {matching_pnos}") # 📋 for cell P


Total Pnos compared: 10
Matching on all codes: 3 (30.0%)
Pnos matching on all codes: [66, 173, 210]
